In [ ]:
import re
import spacy
import joblib
import pandas as pd
import numpy as np
from datetime import datetime
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertModel
import torch
import nltk

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# ***Extracting Some features from Resume and Job Description and Deriving one metric , then Using Kmeans Clustering for Resume Screening***

In [ ]:
new_df = pd.read_excel('/content/processed_data.xlsx')

In [ ]:
new_df.head()

,Unnamed: 0.1,Unnamed: 0,ID,Name,Role,Transcript,Resume,decision,Reason for decision,Job Description,...,resume_transcript_similarity,resume_sentence_count,resume_avg_word_length,skill_match_count,university_education_count,transcript_vocab_diversity,transcript_avg_sentence_length,resume_jd_similarity_transformers,transcript_jd_similarity_transformers,transcript_resume_similarity_transformers
0,0,0,brenbr359,brent brown,product manager,product manager interview transcript\n\ninterv...,here's a sample resume for brent brown applyin...,select,experience,we are looking for a skilled product manager w...,...,0.226753,1,7.737557,8,2,0.548851,348,0.523329,0.490810,0.726645
1,1,1,jameay305,james ayala,software engineer,software engineer interview transcript\n\ninte...,here's a sample resume for james ayala applyin...,select,experience,we are looking for a skilled software engineer...,...,0.169317,1,7.676349,7,2,0.591731,387,0.356570,0.290567,0.698974
2,2,2,scotri565,scott rivera,data engineer,here is a simulated interview for scott rivera...,here's a sample resume for scott rivera applyi...,reject,experience,we are looking for a skilled data engineer wit...,...,0.346429,1,7.117647,9,1,0.475177,282,0.661279,0.487049,0.749828
3,3,3,emilke232,emily kelly,ui engineer,interview transcript: emily kelly for ui engin...,here's a sample resume for emily kelly:\n\nemi...,select,experience,we are looking for a skilled ui engineer with ...,...,0.235884,1,7.130802,7,1,0.519900,402,0.450366,0.219879,0.653381
4,4,4,ashlra638,ashley ray,data scientist,data scientist interview transcript\n\ncompany...,here's a sample resume for ashley ray applying...,reject,cultural fit,we are looking for a skilled data scientist wi...,...,0.180783,1,7.394649,9,2,0.522876,306,0.486396,0.384930,0.665007


In [ ]:
new_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'ID', 'Name', 'Role', 'Transcript',
       'Resume', 'decision', 'Reason for decision', 'Job Description',
       'Transcript_processed', 'Resume_processed', 'Job_Description_processed',
       'Transcript_sentiment', 'Resume_sentiment', 'JobDescription_sentiment',
       'Transcript_words', 'Resume_words', 'Years_Experience',
       'resume_jd_similarity', 'Transcript_jd_similarity',
       'resume_transcript_similarity', 'resume_sentence_count',
       'resume_avg_word_length', 'skill_match_count',
       'university_education_count', 'transcript_vocab_diversity',
       'transcript_avg_sentence_length', 'resume_jd_similarity_transformers',
       'transcript_jd_similarity_transformers',
       'transcript_resume_similarity_transformers'],
      dtype='object')

In [ ]:
req_df = new_df[['Role','decision','Resume_sentiment' ,'Resume_words' , 'Years_Experience' , 'resume_jd_similarity' ,  'resume_sentence_count' , 'resume_avg_word_length' ,  'skill_match_count' , 'university_education_count']]

In [ ]:
req_df.head()

,Role,decision,Resume_sentiment,Resume_words,Years_Experience,resume_jd_similarity,resume_sentence_count,resume_avg_word_length,skill_match_count,university_education_count
0,product manager,select,0.307407,221,14,0.268642,1,7.737557,8,2
1,software engineer,select,0.130833,241,11,0.238920,1,7.676349,7,2
2,data engineer,reject,0.202970,255,14,0.375988,1,7.117647,9,1
3,ui engineer,select,0.131515,237,7,0.248950,1,7.130802,7,1
4,data scientist,reject,0.111750,299,0,0.307717,1,7.394649,9,2


In [ ]:
req_df.columns

Index(['Role', 'decision', 'Resume_sentiment', 'Resume_words',
       'Years_Experience', 'resume_jd_similarity', 'resume_sentence_count',
       'resume_avg_word_length', 'skill_match_count',
       'university_education_count'],
      dtype='object')

In [ ]:
# Drop irrelevant columns (e.g., Role and decision)
df = req_df.drop(columns=['Role', 'decision'], errors='ignore')

# Normalize numerical features
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(df)

# Combine normalized features into a DataFrame
normalized_df = pd.DataFrame(normalized_features, columns=df.columns)

# Calculate a "Fit Score" using a weighted combination of features
# Assigning arbitrary weights (tune as needed)
weights = {
    'Resume_sentiment': 0.2,
    'Resume_words': 0.1,
    'Years_Experience': 0.3,
    'resume_jd_similarity': 0.3,
    'resume_sentence_count': 0.05,
    'resume_avg_word_length': 0.05,
    'skill_match_count': 0.4,
    'university_education_count': 0.2
}

# Create a weighted score
normalized_df['Fit_Score'] = sum(normalized_df[col] * weight for col, weight in weights.items() if col in normalized_df)

# Set thresholds using clustering (optional)
kmeans = KMeans(n_clusters=3, random_state=42)
normalized_df['Suitability_Cluster'] = kmeans.fit_predict(normalized_df[['Fit_Score']])

# Map clusters to meaningful labels (e.g., 0 = Not Suitable, 1 = Moderately Suitable, 2 = Highly Suitable)
cluster_mapping = {0: "Not Suitable", 1: "Moderately Suitable", 2: "Highly Suitable"}
normalized_df['Suitability_Label'] = normalized_df['Suitability_Cluster'].map(cluster_mapping)

# Display results
print(normalized_df[['Fit_Score', 'Suitability_Label']].head())

# Save results to a new CSV
normalized_df.to_csv('resume_screening_results.csv', index=False)


   Fit_Score Suitability_Label
0   0.526448   Highly Suitable
1   0.465071   Highly Suitable
2   0.472942   Highly Suitable
3   0.379833   Highly Suitable
4   0.482700   Highly Suitable


In [ ]:
normalized_df.Suitability_Label.value_counts()

,count
Suitability_Label,
Highly Suitable,1309
Not Suitable,1125
Moderately Suitable,524


In [ ]:
normalized_df['Role'] = req_df['Role']
normalized_df['decision'] = req_df['decision']

In [ ]:
normalized_df.head()

,Resume_sentiment,Resume_words,Years_Experience,resume_jd_similarity,resume_sentence_count,resume_avg_word_length,skill_match_count,university_education_count,Fit_Score,Suitability_Cluster,Suitability_Label,Role,decision
0,0.752028,0.439024,0.090323,0.342752,0.0,0.682026,0.086957,0.666667,0.526448,2,Highly Suitable,product manager,select
1,0.533413,0.493225,0.070968,0.304832,0.0,0.651164,0.076087,0.666667,0.465071,2,Highly Suitable,software engineer,select
2,0.622724,0.531165,0.090323,0.479712,0.0,0.369455,0.097826,0.333333,0.472942,2,Highly Suitable,data engineer,reject
3,0.534257,0.482385,0.045161,0.317628,0.0,0.376088,0.076087,0.333333,0.379833,2,Highly Suitable,ui engineer,select
4,0.509786,0.650407,0.000000,0.392607,0.0,0.509125,0.097826,0.666667,0.482700,2,Highly Suitable,data scientist,reject


In [ ]:
bins = [0,0.1, 0.2, 0.4, 0.6, 0.8, 1]
labels = ['0-0.1','0.1-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1']

# Create a new column with binned values
normalized_df['similarity_bin'] = pd.cut(normalized_df['Fit_Score'], bins=bins, labels=labels, include_lowest=True)

# ***Using Normal Resume and Job Description Similarity for Resume Screening***

In [ ]:
# Use a single TfidfVectorizer for both Resume and Job Description
tfidf_vectorizer4 = TfidfVectorizer()
combined_text = new_df['Resume_processed'] + " " + new_df['Job_Description_processed']
tfidf_vectorizer4.fit(combined_text)

# Split the combined TF-IDF matrix back into Resume and Job Description matrices
resume_matrix = tfidf_vectorizer4.transform(new_df['Resume_processed'])
jobDescription_matrix = tfidf_vectorizer4.transform(new_df['Job_Description_processed'])

# Calculate row-wise cosine similarity
cos_rjd = []
for i in range(new_df.shape[0]):
    cos_rjd.append(cosine_similarity(resume_matrix[i], jobDescription_matrix[i])[0][0])

In [ ]:
new_df['resume_job_similarity'] = cos_rjd

In [ ]:
## Here , Except for data engineer , software engineer , and ui engineer for all other roles for rejection the similarity between job description and resume is higher

In [ ]:
bins = [0,0.1, 0.2, 0.4, 0.6, 0.8, 1]
labels = ['0-0.1','0.1-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1']

# Create a new column with binned values
new_df['similarity_bin'] = pd.cut(new_df['resume_job_similarity'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
new_df.groupby(['similarity_bin', 'decision','Role']).size().unstack(fill_value=0)

<ipython-input-16-57c296a99497>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  new_df.groupby(['similarity_bin', 'decision','Role']).size().unstack(fill_value=0)


Role                     data analyst  data engineer  data scientist  \
similarity_bin decision                                                
0-0.1          reject              81             89              79   
               select              96             85              73   
0.1-0.2        reject              51             42              86   
               select              26             24              76   
0.2-0.4        reject               4             81             116   
               select               3             70             118   
0.4-0.6        reject               3             57              23   
               select               3             60              21   
0.6-0.8        reject               0              1               0   
               select               0              0               0   
0.8-1          reject               0              0               0   
               select               0              0               0   

Role                     product manager  software engineer  ui designer  \
similarity_bin decision                                                    
0-0.1          reject                 54                 84           68   
               select                 84                 82           76   
0.1-0.2        reject                 41                 62           44   
               select                 14                 33           27   
0.2-0.4        reject                 82                129           13   
               select                 83                149            6   
0.4-0.6        reject                 67                  5            0   
               select                 46                  7            0   
0.6-0.8        reject                 17                  0            0   
               select                 26                  0            0   
0.8-1          reject                  0                  0            0   
               select                  0                  0            0   

Role                     ui engineer  
similarity_bin decision               
0-0.1          reject              0  
               select              0  
0.1-0.2        reject              5  
               select              0  
0.2-0.4        reject             87  
               select             92  
0.4-0.6        reject             52  
               select             54  
0.6-0.8        reject              0  
               select              1  
0.8-1          reject              0  
               select              0

## From this For Tf-Idf embeddings we can see that,

1. For Data Engineer We can see that , if similarity between Job description and Resume is less than the 0.2 then we are able to reject the application
2. Same for Product Manager , software Engineer , ui enginner, data analyst , data scientist
3. For ui designer 0.4 will be a good threshold for rejection

In [ ]:
## Using Bert Embeddings

In [ ]:
df = pd.read_excel('/content/bert_embeddings.xlsx')

In [ ]:
df_expanded = df.copy()

In [ ]:
tra_emb = [col for col in df_expanded.columns if col.startswith('trans_emb_')]
resume_emb = [col for col in df_expanded.columns if col.startswith('resume_emb_')]
jd_emb = [col for col in df_expanded.columns if col.startswith('jd_emb_')]

In [ ]:
tra_emb = df_expanded[tra_emb].values
resume_emb = df_expanded[resume_emb].values
jd_emb = df_expanded[jd_emb].values

In [ ]:
jd_res_simi = []
for jd , res in zip(jd_emb , resume_emb):
    jd_res_simi.append(cosine_similarity([jd],[res])[0][0])

In [ ]:
df_expanded['Bert_ResJobDesc_Similarity'] = jd_res_simi

In [ ]:
bins = [ 0.95, 0.955, 0.96, 0.965, 0.97, 0.975, 0.98, 0.985, 0.99, 0.995, 1]
labels =['0.95-0.955', '0.955-0.96', '0.96-0.965', '0.965-0.97', '0.97-0.975', '0.975-0.98', '0.98-0.985', '0.985-0.99', '0.99-0.995', '0.995-1']

# Create a new column with binned values
df_expanded['similarity_bin'] = pd.cut(df_expanded['Bert_ResJobDesc_Similarity'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
df_expanded.groupby(['similarity_bin', 'decision','Role']).size().unstack(fill_value=0)

<ipython-input-33-8d78e5439a6d>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_expanded.groupby(['similarity_bin', 'decision','Role']).size().unstack(fill_value=0)


Role                     data analyst  data engineer  data scientist  \
similarity_bin decision                                                
0.95-0.955     reject               0              3               9   
               select               1              7               6   
0.955-0.96     reject               0             24              16   
               select               1             14              19   
0.96-0.965     reject               0             11              13   
               select               0             15              13   
0.965-0.97     reject               3              1               1   
               select               1              5               1   
0.97-0.975     reject               1              0               2   
               select               1              1               0   
0.975-0.98     reject               1              0               0   
               select               1              0               1   
0.98-0.985     reject               0              0               0   
               select               0              0               0   
0.985-0.99     reject               0              0               0   
               select               0              0               0   
0.99-0.995     reject               0              0               0   
               select               0              0               0   
0.995-1        reject               0              0               0   
               select               0              0               0   

Role                     product manager  software engineer  ui designer  \
similarity_bin decision                                                    
0.95-0.955     reject                  2                 13            0   
               select                  6                 10            0   
0.955-0.96     reject                 12                 20            0   
               select                  4                 20            0   
0.96-0.965     reject                 20                 13            0   
               select                 17                 22            0   
0.965-0.97     reject                 16                  3            0   
               select                 14                  9            0   
0.97-0.975     reject                  5                  0            0   
               select                  7                  2            0   
0.975-0.98     reject                  0                  0            0   
               select                  1                  1            0   
0.98-0.985     reject                  0                  0            0   
               select                  0                  0            0   
0.985-0.99     reject                  0                  0            0   
               select                  0                  0            0   
0.99-0.995     reject                  0                  0            0   
               select                  0                  0            0   
0.995-1        reject                  0                  0            0   
               select                  0                  0            0   

Role                     ui engineer  
similarity_bin decision               
0.95-0.955     reject              1  
               select              0  
0.955-0.96     reject              1  
               select              1  
0.96-0.965     reject             15  
               select              9  
0.965-0.97     reject             25  
               select             22  
0.97-0.975     reject             11  
               select             15  
0.975-0.98     reject              0  
               select              1  
0.98-0.985     reject              0  
               select              0  
0.985-0.99     reject              0  
               select              0  
0.99-0.995     reject              0  
 

#### Here using the BERT embeddings we can see that, if similarity score falls between 0.95-0.96, we can reject for almost all positions